In [5]:
%load_ext nb_black
%load_ext lab_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black
The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


<IPython.core.display.Javascript object>

In [6]:
import pathlib
import datetime

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
import datetime

from tensorflow import keras
from tensorflow.keras import layers

from keras.callbacks import ModelCheckpoint
import keras_tuner as kt

# 60%
PATH_TREINO = "../Data/3_Gold/Treino_all_stocks.csv"
DF_TREINO = pd.read_csv(PATH_TREINO, sep=",")
train_dataset = DF_TREINO
train_dataset = train_dataset.drop(
    [
        "oil_5",
        "usd_5",
        "abev_5",
        "jbs_5",
        "petr_5",
        "vale_5",
    ],
    axis=1,
)
train_labels = train_dataset.pop("ibova_5")

# 20%
PATH_VALIDACAO = "../Data/3_Gold/Validacao_all_stocks.csv"
DF_VALIDACAO = pd.read_csv(PATH_VALIDACAO, sep=",")
valid_dataset = DF_VALIDACAO
valid_dataset = valid_dataset.drop(
    [
        "ibova_5",
        "oil_5",
        "usd_5",
        "abev_5",
        "jbs_5",
        "petr_5",
        "vale_5",
    ],
    axis=1,
)
# valid_labels = valid_dataset.pop('ibova_5')

# 20%
PATH_TESTE = "../Data/3_Gold/Teste_all_stocks.csv"
DF_TESTE = pd.read_csv(PATH_TESTE, sep=",")
test_dataset = DF_TESTE
test_dataset = test_dataset.drop(
    [
        "oil_5",
        "usd_5",
        "abev_5",
        "jbs_5",
        "petr_5",
        "vale_5",
    ],
    axis=1,
)
test_labels = test_dataset.pop("ibova_5")

"""
	Resultado
"""
list_activation = []
list_best_epoch = []
list_best_model_path = []
list_hp_unit_1 = []
list_hp_unit_2 = []
list_hp_unit_3 = []
list_hp_unit_4 = []
list_kernel_size_1 = []
list_kernel_size_2 = []
list_learning_rate = []
list_n_filter_1 = []
list_n_filter_2 = []
list_val_mae = []
list_val_mse = []

<IPython.core.display.Javascript object>

In [7]:
TYPE_MODEL = "CNN"
EPOCHS = 500


def repetir():
    def model_builder(hp):
        hp_filter_1 = hp.Int("n_filter_1", min_value=4, max_value=16, step=1)
        hp_kernel_size_1 = hp.Int(
            "kernel_size_1", min_value=1, max_value=15, step=2
        )  # precisa ser impar

        activationL = [
            "relu",
            "sigmoid",
            "softplus",
            "softsign",
            "tanh",
            "selu",
            "elu",
            "exponential",
        ]
        activation_choice = hp.Choice("activation", values=activationL)
        hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])

        model = keras.Sequential(
            [
                tf.keras.layers.Conv1D(
                    input_shape=[len(train_dataset.keys()), 1],
                    filters=hp_filter_1,
                    kernel_size=hp_kernel_size_1,
                    activation=activation_choice,  # overkill - relu ou elu
                ),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(units=1, activation="linear"),
            ]
        )

        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
            loss="mse",
            metrics=["mae", "mse"],
        )

        return model

    """
    =============================================
    Executing:
    =============================================
    """
    tuner = kt.Hyperband(
        model_builder,
        objective="val_mse",
        max_epochs=50,
        factor=3,
        directory="logs",
        project_name="hyper_parameters",
    )
    # Build the model with the optimal hyperparameters and train it on the data for 50 epochs
    save_time = datetime.datetime.strftime(datetime.datetime.now(), "%y_%m_%d_%Hh%Mm%S")

    bst_model_path = f"../Model/Models/{TYPE_MODEL}/model_{save_time}.h5"
    model_checkpoint = ModelCheckpoint(
        bst_model_path, save_best_only=True, monitor="val_mse", mode="min"
    )

    best_hps = tuner.get_best_hyperparameters(num_trials=100)[0]

    model = tuner.hypermodel.build(best_hps)
    history = model.fit(
        train_dataset,
        train_labels,
        epochs=EPOCHS,
        validation_split=0.2,
        verbose=0,
        callbacks=([model_checkpoint]),
    )

    # Select the best epoch,
    val_mse_per_epoch = history.history["val_mse"]
    best_epoch = val_mse_per_epoch.index(min(val_mse_per_epoch)) + 1

    try:
        value_activation = best_hps.get("activation")
    except:
        value_activation = None
    try:
        value_learning_rate = best_hps.get("learning_rate")
    except:
        value_learning_rate = None
    try:
        value_unit_1 = best_hps.get("unit_1")
    except:
        value_unit_1 = None
    try:
        value_unit_2 = best_hps.get("unit_2")
    except:
        value_unit_2 = None
    try:
        value_unit_3 = best_hps.get("unit_3")
    except:
        value_unit_3 = None
    try:
        value_unit_4 = best_hps.get("unit_4")
    except:
        value_unit_4 = None
    try:
        value_kernel_size_1 = best_hps.get("kernel_size_1")
    except:
        value_kernel_size_1 = None
    try:
        value_kernel_size_2 = best_hps.get("kernel_size_2")
    except:
        value_kernel_size_2 = None
    try:
        value_n_filter_1 = best_hps.get("n_filter_1")
    except:
        value_n_filter_1 = None
    try:
        value_n_filter_2 = best_hps.get("n_filter_2")
    except:
        value_n_filter_2 = None

    """"Repeticao MODEL.ipynb"""

    def save_in_list(one_list, value=None):
        try:
            one_list.append(value)
            return one_list
        except:
            one_list.append(None)
            return one_list

    save_in_list(list_activation, value_activation)
    save_in_list(list_best_epoch, best_epoch)
    save_in_list(list_best_model_path, bst_model_path)
    save_in_list(list_hp_unit_1, value_unit_1)
    save_in_list(list_hp_unit_2, value_unit_2)
    save_in_list(list_hp_unit_3, value_unit_3)
    save_in_list(list_hp_unit_4, value_unit_4)
    save_in_list(list_kernel_size_1, value_kernel_size_1)
    save_in_list(list_kernel_size_2, value_kernel_size_2)
    save_in_list(list_learning_rate, value_learning_rate)
    save_in_list(list_n_filter_1, value_n_filter_1)
    save_in_list(list_n_filter_2, value_n_filter_2)
    save_in_list(list_val_mae, min(history.history["val_mae"]))
    save_in_list(list_val_mse, min(history.history["val_mse"]))

<IPython.core.display.Javascript object>

In [8]:
for i in range(2):
    repetir()

IndexError: list index out of range

<IPython.core.display.Javascript object>

In [ ]:
result_columns = [
    "activation",
    "best_epoch",
    "best_model_path",
    "hp_unit_1",
    "hp_unit_2",
    "hp_unit_3",
    "hp_unit_4",
    "kernel_size_1",
    "kernel_size_2",
    "learning_rate",
    "n_filter_1",
    "n_filter_2",
    "val_mae",
    "val_mse",
]

data = list(
    zip(
        list_activation,
        list_best_epoch,
        list_best_model_path,
        list_hp_unit_1,
        list_hp_unit_2,
        list_hp_unit_3,
        list_hp_unit_4,
        list_kernel_size_1,
        list_kernel_size_2,
        list_learning_rate,
        list_n_filter_1,
        list_n_filter_2,
        list_val_mae,
        list_val_mse,
    )
)

df_resultados = pd.DataFrame(
    data,
    columns=result_columns,
)

TYPE_MODEL = "CNN"
df_resultados.to_csv(f"../Results/Resultados_{TYPE_MODEL}.csv")
df_resultados

,activation,best_epoch,best_model_path,hp_unit_1,hp_unit_2,hp_unit_3,hp_unit_4,kernel_size_1,kernel_size_2,learning_rate,n_filter_1,n_filter_2,val_mae,val_mse
0,exponential,1,Models/CNN/model_22_10_09_19h12m41.h5,59,25,37,8,1,None,0.0100,4,None,NaN,NaN
1,elu,256,Models/CNN/model_22_10_09_19h13m37.h5,50,19,12,21,1,None,0.0100,4,None,0.208025,0.070775
2,exponential,1,Models/CNN/model_22_10_09_19h14m28.h5,41,23,29,34,1,None,0.0001,4,None,NaN,NaN
3,elu,286,Models/CNN/model_22_10_09_19h15m13.h5,50,19,12,21,1,None,0.0100,4,None,0.208909,0.071108
4,elu,318,Models/CNN/model_22_10_09_19h16m05.h5,50,19,12,21,1,None,0.0100,4,None,0.208090,0.069598
5,elu,263,Models/CNN/model_22_10_09_19h16m57.h5,50,19,12,21,1,None,0.0100,4,None,0.206372,0.069183
6,elu,471,Models/CNN/model_22_10_09_19h17m52.h5,50,19,12,21,1,None,0.0100,4,None,0.207422,0.071735
7,softsign,461,Models/CNN/model_22_10_09_19h18m47.h5,62,59,47,27,1,None,0.0001,4,None,0.221414,0.080946
8,exponential,1,Models/CNN/model_22_10_09_19h19m36.h5,59,25,37,8,1,None,0.0100,4,None,NaN,NaN
9,elu,290,Models/CNN/model_22_10_09_19h20m24.h5,50,19,12,21,1,None,0.0100,4,None,0.209214,0.071193


<IPython.core.display.Javascript object>